########## choose stock list (aka market) ##################

In [12]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import date

In [13]:
## To acquire stock list in NASDAQ ###
ex1 = pd.read_excel('stock_info_hk.xlsx', sheet_name='ListOfSecurities',skiprows=2,dtype=str)
ex1 = ex1.loc[ex1['Category']=='Equity']
ex1 = ex1.loc[ex1['Sub-Category']=='Equity Securities (Main Board)']
ex1 = ex1.loc[ex1['Trading Currency']=='HKD']
ex1['Stock Code'] = ex1['Stock Code'].str.slice(1)

df = pd.DataFrame(ex1['Stock Code'].astype('str') + '.HK')
df = df.rename(columns={'Stock Code' : 'ticker'})
df = df.reset_index(drop=True)
df = df.drop_duplicates()
df = df.reset_index(drop=True)
df

,ticker
0,0001.HK
1,0002.HK
2,0003.HK
3,0004.HK
4,0005.HK
...,...
2278,9995.HK
2279,9996.HK
2280,9997.HK
2281,9998.HK


### Acquire informations from yfinance function >> .info

In [14]:
# 1st pulling date for this data #
df['date_pulling'] = date.today()

In [15]:
# 2nd acquiring information from .info #
info_attribute_list = [
    'industry',
    'sector',
    'enterpriseValue',
    'totalCashPerShare'
]
# 3rd acquiring information from .quarterly_balance_sheet #
balancesheet_list = [
    'Total Assets',
    'Current Liabilities'
]

# 4th acquiring information from .quarterly_financials #
ttm = 4
financials_list = [
    'EBIT',
    'Operating Income'
]

info_attribute_list_buffer = []
balancesheet_list_buffer = []
financials_list_buffer = []
for i in range(len(df)):
    # print(i, " ",df['ticker'][i])
    yfticker = yf.Ticker(df['ticker'][i])

    # 2nd acquiring information from .info #
    info_attribute_list_buffer_r = []
    for j in range(len(info_attribute_list)):
        try:
            info_attribute_list_buffer_r.append(yfticker.info[info_attribute_list[j]])
        except:
            info_attribute_list_buffer_r.append(None)
    info_attribute_list_buffer.append(info_attribute_list_buffer_r)

    # 3rd acquiring information from .balancesheet #
    balancesheet_list_buffer_r = []
    for j in range(len(balancesheet_list)):
        try:
            balancesheet_list_buffer_r.append(yfticker.balance_sheet.loc[yfticker.balance_sheet.index == balancesheet_list[j]].values[0][0])
        except:
            balancesheet_list_buffer_r.append(None)
    balancesheet_list_buffer.append(balancesheet_list_buffer_r)

    # 4th acquiring information from .financials #
    financials_list_buffer_r = []
    for j in range(len(financials_list)):
        try:
            financials_list_buffer_r.append(yfticker.financials.loc[yfticker.financials.index == financials_list[j]].values[0][0])
        except:
            financials_list_buffer_r.append(None)
    try:
        financials_list_buffer_r.append(yfticker.financials.columns[0])
    except:
        financials_list_buffer_r.append(None)
    financials_list_buffer.append(financials_list_buffer_r)

financials_list.append('ttm_latest')
df = df.join(pd.DataFrame(info_attribute_list_buffer, columns=info_attribute_list))
df = df.join(pd.DataFrame(balancesheet_list_buffer, columns=balancesheet_list))
df = df.join(pd.DataFrame(financials_list_buffer, columns=financials_list))


## Drop NAN #############
df = df.dropna()
#### Drop where marketcap and EBIT < 0#############
df = df.loc[df['enterpriseValue'] >0]
df = df.loc[df['EBIT'] >0]
df = df.loc[df['Operating Income'] >0]
df = df.reset_index(drop=True)

df.to_csv('data_stock_hk.csv',index= False)

############################################################################

In [16]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import date
df = pd.read_csv('data_stock_hk.csv')
print('sector', df['sector'].unique())
print('industry', df['industry'].unique())

sector ['Industrials' 'Utilities' 'Real Estate' 'Communication Services'
 'Healthcare' 'Consumer Cyclical' 'Technology' 'Financial Services'
 'Basic Materials' 'Energy' 'Consumer Defensive']
industry ['Conglomerates' 'Utilities - Regulated Electric'
 'Utilities - Regulated Gas' 'Real Estate - Development'
 'Utilities - Independent Power Producers' 'Telecom Services'
 'Real Estate Services' 'Real Estate - Diversified'
 'Drug Manufacturers - Specialty & Generic' 'Publishing'
 'Resorts & Casinos' 'Electronic Components'
 'Farm & Heavy Construction Machinery' 'Lodging'
 'Information Technology Services' 'Restaurants'
 'Specialty Industrial Machinery' 'Railroads' 'Semiconductors'
 'Utilities - Renewable' 'Credit Services' 'Auto Parts' 'Aluminum'
 'Consumer Electronics' 'Steel' 'Specialty Retail'
 'Infrastructure Operations' 'Capital Markets' 'Luxury Goods'
 'Oil & Gas Refining & Marketing' 'Entertainment' 'Packaged Foods'
 'Marine Shipping' 'Textile Manufacturing'
 'Internet Content & Infor

In [17]:

### Next I perform calculations for MF_ROC and MF_EY ###
represents_earning = 'Operating Income'
# represents_earning = 'EBIT'

df['MF_ROC'] = df[represents_earning]/(df['Total Assets'] - df['Current Liabilities'])
df['MF_EY'] = df[represents_earning]/df['enterpriseValue']
df

,ticker,date_pulling,industry,sector,enterpriseValue,totalCashPerShare,Total Assets,Current Liabilities,EBIT,Operating Income,ttm_latest,MF_ROC,MF_EY
0,0001.HK,2024-08-19,Conglomerates,Industrials,4.821852e+11,34.370,1.158903e+12,1.625250e+11,4.506100e+10,4.641900e+10,2023-12-31,0.046588,0.096268
1,0002.HK,2024-08-19,Utilities - Regulated Electric,Utilities,2.394790e+11,2.160,2.290510e+11,4.247900e+10,1.247200e+10,1.518400e+10,2023-12-31,0.081384,0.063404
2,0003.HK,2024-08-19,Utilities - Regulated Gas,Utilities,1.826674e+11,0.559,1.619776e+11,3.942450e+10,1.138900e+10,8.137300e+09,2023-12-31,0.066398,0.044547
3,0004.HK,2024-08-19,Real Estate - Development,Real Estate,7.606260e+10,3.478,2.048770e+11,2.676300e+10,3.044000e+09,6.407000e+09,2023-12-31,0.035971,0.084233
4,0006.HK,2024-08-19,Utilities - Independent Power Producers,Utilities,1.164617e+11,0.696,9.570200e+10,3.249000e+09,6.395000e+09,1.131000e+09,2023-12-31,0.012233,0.009711
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1066,9992.HK,2024-08-19,Leisure,Consumer Cyclical,4.680557e+10,4.493,9.968863e+09,1.733301e+09,1.448092e+09,1.226965e+09,2023-12-31,0.148984,0.026214
1067,9993.HK,2024-08-19,Real Estate—Development,Real Estate,4.883974e+10,1.754,1.297780e+11,7.600156e+10,9.935970e+08,8.170440e+08,2023-12-31,0.015193,0.016729
1068,9997.HK,2024-08-19,Medical Instruments & Supplies,Healthcare,4.437055e+09,2.210,4.263818e+09,1.908240e+08,5.910280e+08,5.009780e+08,2023-12-31,0.123000,0.112908
1069,9998.HK,2024-08-19,Engineering & Construction,Industrials,5.968200e+07,0.049,8.529300e+07,4.563800e+07,2.525000e+06,1.326000e+06,2023-06-30,0.033438,0.022218


In [18]:
### Drop Utilities, Energy, and Financial Services  as suggested from the book ###
sectortoexclude = [
    'Utilities',
    'Energy',
    'Financial Services',
    'Real Estate'
]
for i in sectortoexclude:
    print('exclude sector > ',i)
    try:
         
        df = df.loc[df['sector'] != i]
    except:
        None
        
industrytoexclude =[
    'Engineering & Construction',
    'Building Products & Equipment',
    'Building Materials'
]
for i in industrytoexclude:
    print('exclude industry > ',i)
    try:
         
        df = df.loc[df['industry'] != i]
    except:
        None

df = df.reset_index(drop=True)

exclude sector >  Utilities
exclude sector >  Energy
exclude sector >  Financial Services
exclude sector >  Real Estate
exclude industry >  Engineering & Construction
exclude industry >  Building Products & Equipment
exclude industry >  Building Materials


In [19]:
#### Select market size by choosing ' market ' ###
market =  50000000 * 7.79 ### in HKD
df_market = df.loc[df['enterpriseValue'] >= market]
df_market = df_market.reset_index(drop=True)

In [20]:
### Ranking regarding MFs####

df_market['Ranking_MF_ROC'] = df_market['MF_ROC'].rank()
df_market['Ranking_MF_EY'] = df_market['MF_EY'].rank()
df_market['Ranking_MF'] = df_market['Ranking_MF_ROC'] + df_market['Ranking_MF_EY']

df_market.loc[df_market['Ranking_MF'] == df_market['Ranking_MF'].max()]

,ticker,date_pulling,industry,sector,enterpriseValue,totalCashPerShare,Total Assets,Current Liabilities,EBIT,Operating Income,ttm_latest,MF_ROC,MF_EY,Ranking_MF_ROC,Ranking_MF_EY,Ranking_MF
594,9696.HK,2024-08-19,Other Industrial Metals & Mining,Basic Materials,3.976231e+10,5.731,7.496907e+10,6.659867e+09,3.681243e+10,3.389577e+10,2023-12-31,0.496211,0.85246,620.0,617.0,1237.0


In [21]:
numstocks = 50
df_sorted = df_market.sort_values(by=['Ranking_MF'],ascending=False)
df_sorted = df_sorted.reset_index(drop=True)
df_sorted[:numstocks]

,ticker,date_pulling,industry,sector,enterpriseValue,totalCashPerShare,Total Assets,Current Liabilities,EBIT,Operating Income,ttm_latest,MF_ROC,MF_EY,Ranking_MF_ROC,Ranking_MF_EY,Ranking_MF
0,9696.HK,2024-08-19,Other Industrial Metals & Mining,Basic Materials,3.976231e+10,5.731,7.496907e+10,6.659867e+09,3.681243e+10,3.389577e+10,2023-12-31,0.496211,0.852460,620.0,617.0,1237.0
1,2660.HK,2024-08-19,Electronic Gaming & Multimedia,Communication Services,9.326287e+08,1.929,2.640490e+09,2.901240e+08,8.836770e+08,8.373140e+08,2023-12-31,0.356248,0.897800,609.0,619.0,1228.0
2,0085.HK,2024-08-19,Semiconductors,Technology,9.370607e+08,0.883,4.326786e+09,1.754651e+09,7.792590e+08,7.169560e+08,2023-12-31,0.278740,0.765112,595.0,616.0,1211.0
3,1958.HK,2024-08-19,Auto Manufacturers,Consumer Cyclical,1.564219e+10,4.395,1.687232e+11,7.645346e+10,2.149504e+10,2.347360e+10,2023-12-31,0.254402,1.500659,587.0,621.0,1208.0
4,1184.HK,2024-08-19,Electronics & Computer Distribution,Technology,2.020398e+09,2.668,7.825902e+09,4.872046e+09,8.026060e+08,9.144950e+08,2023-12-31,0.309594,0.452631,601.0,603.0,1204.0
5,9919.HK,2024-08-19,Advertising Agencies,Communication Services,5.016871e+08,0.412,7.954010e+08,4.287720e+08,1.651950e+08,1.620090e+08,2023-12-31,0.441888,0.322928,619.0,581.0,1200.0
6,1126.HK,2024-08-19,Leisure,Consumer Cyclical,1.705538e+09,2.056,4.384667e+09,1.126744e+09,8.441880e+08,8.092650e+08,2022-12-31,0.248399,0.474492,581.0,605.0,1186.0
7,1558.HK,2024-08-19,Drug Manufacturers—Specialty & Generic,Healthcare,9.451915e+09,1.924,1.274433e+10,4.332220e+09,2.354169e+09,2.877598e+09,2023-12-31,0.342078,0.304446,607.0,572.0,1179.0
8,3991.HK,2024-08-19,Electronics & Computer Distribution,Technology,1.721864e+09,0.470,1.740746e+10,1.458646e+10,6.771960e+08,6.742340e+08,2023-12-31,0.239005,0.391572,576.0,595.0,1171.0
9,1003.HK,2024-08-19,Entertainment,Communication Services,1.681311e+09,0.085,2.234243e+09,7.771260e+08,2.170080e+08,4.671600e+08,2023-12-31,0.320606,0.277855,604.0,564.0,1168.0


In [22]:
df_sorted_2 = df_sorted[:numstocks].copy()
df_sorted_2 = df_sorted_2.sort_values(by=['totalCashPerShare'],ascending=False)
df_sorted_2 = df_sorted_2.reset_index(drop=True)
df_sorted_2

,ticker,date_pulling,industry,sector,enterpriseValue,totalCashPerShare,Total Assets,Current Liabilities,EBIT,Operating Income,ttm_latest,MF_ROC,MF_EY,Ranking_MF_ROC,Ranking_MF_EY,Ranking_MF
0,0921.HK,2024-08-19,"Furnishings, Fixtures & Appliances",Consumer Cyclical,1.567067e+10,15.881,6.594650e+10,4.404209e+10,5.828946e+09,4.537223e+09,2023-12-31,0.207137,0.289536,557.0,568.0,1125.0
1,2877.HK,2024-08-19,Drug Manufacturers—Specialty & Generic,Healthcare,5.677010e+08,7.827,9.573993e+09,2.433718e+09,1.242990e+09,9.755350e+08,2023-12-31,0.136624,1.718396,456.0,623.0,1079.0
2,0098.HK,2024-08-19,Aluminum,Basic Materials,3.437665e+09,6.752,1.286623e+10,4.794935e+09,1.054379e+09,1.291607e+09,2023-12-31,0.160025,0.375722,505.0,590.0,1095.0
3,9696.HK,2024-08-19,Other Industrial Metals & Mining,Basic Materials,3.976231e+10,5.731,7.496907e+10,6.659867e+09,3.681243e+10,3.389577e+10,2023-12-31,0.496211,0.852460,620.0,617.0,1237.0
4,1681.HK,2024-08-19,Drug Manufacturers—Specialty & Generic,Healthcare,1.697790e+09,4.522,5.507994e+09,1.494996e+09,8.849670e+08,8.226300e+08,2023-12-31,0.204991,0.484530,555.0,606.0,1161.0
5,1958.HK,2024-08-19,Auto Manufacturers,Consumer Cyclical,1.564219e+10,4.395,1.687232e+11,7.645346e+10,2.149504e+10,2.347360e+10,2023-12-31,0.254402,1.500659,587.0,621.0,1208.0
6,1797.HK,2024-08-19,Education & Training Services,Consumer Defensive,7.399076e+09,3.435,3.852938e+09,1.028829e+09,1.174403e+09,1.099810e+09,2023-05-31,0.389436,0.148642,613.0,466.0,1079.0
7,3306.HK,2024-08-19,Apparel Manufacturing,Consumer Cyclical,5.576069e+09,3.221,4.065827e+09,1.615785e+09,8.961580e+08,8.412530e+08,2023-06-30,0.343363,0.150868,608.0,471.0,1079.0
8,1651.HK,2024-08-19,Tools & Accessories,Industrials,2.577940e+09,2.943,3.632235e+09,8.062080e+08,7.037270e+08,6.817760e+08,2024-03-31,0.241249,0.264465,577.0,560.0,1137.0
9,0546.HK,2024-08-19,Specialty Chemicals,Basic Materials,1.393601e+10,2.756,3.256089e+10,1.404456e+10,4.015298e+09,3.491442e+09,2023-12-31,0.188560,0.250534,543.0,556.0,1099.0
